# Optuna in RayTune

This notebook first explores Optuna's integration into RayTune,
following the tutorial here:

https://docs.ray.io/en/latest/tune/examples/optuna_example.html

In [1]:
import time
from typing import Dict, Optional, Any

import ray
from ray import train, tune
from ray.tune.search import ConcurrencyLimiter
from ray.tune.search.optuna import OptunaSearch

In [2]:
def evaluate(step, width, height, activation):
    time.sleep(0.1)
    activation_boost = 10 if activation == "relu" else 0
    return (0.1 + width * step / 100) ** (-1) + height * 0.1 + activation_boost

In [3]:
def objective(config):
    """
    Evaluates score of experiment in training loop
    Uses train.report to report score back to Tune
    """
    for step in range(config["steps"]):
        score = evaluate(step, config["width"], config["height"], config["activation"])
        train.report({
            "iterations": step,
            "mean_loss": score
        })

## Define the Search Space

In [12]:
search_space = {
    "steps": 100,
    "width": tune.uniform(0, 20),
    "height": tune.uniform(-100, 100),
    "activation": tune.choice(["relu", "tanh"])
}

In [13]:
algo = OptunaSearch()

In [14]:
# constrain number of concurrent trials to 4
algo = ConcurrencyLimiter(algo, max_concurrent=4)

In [15]:
num_samples = 5

### Optimisation

Run experiment to minimize mean loss of objective by searching search space via `algo`, `num_samples` times.

In [16]:
tuner = tune.Tuner(
    objective,
    tune_config=tune.TuneConfig(
        metric="mean_loss",
        mode="min",
        search_alg=algo,
        num_samples=num_samples,
    ),
    param_space=search_space,
)

results = tuner.fit()

2024-12-03 17:56:44,166	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/Users/sidharrthnagappan/ray_results/objective_2024-12-03_17-56-20' in 0.0100s.
2024-12-03 17:56:44,172	INFO tune.py:1041 -- Total run time: 23.18 seconds (23.15 seconds for the tuning loop).


In [20]:
print("Best hyperparameters found were: ", results.get_best_result().config)

Best hyperparameters found were:  {'steps': 100, 'width': 6.346555967276859, 'height': 39.14944808941103, 'activation': 'tanh'}


### Initial Hyperparameters

You can also provide an initial set of hyperparameters that are promising. 

```python
initial_params = [
    {"width": 1, "height": 2, "activation": "relu"},
    {"width": 4, "height": 2, "activation": "relu"},
]

searcher = OptunaSearch(points_to_evaluate=initial_params)
```



### Multi-Objective Optimisation

In [ ]:
def multi_objective(config):
    width, height = config["width"], config["height"]
    
    for step in range(config["steps"]):
        intermediate_score = evaluate(step, config["width"], config["height"], config["activation"])
        
        train.report({
            "iterations": step,
            "loss": intermediate_score,
            "gain": intermediate_score * xwidth
        })

In [24]:
searcher = OptunaSearch(metric=["loss", "gain"], mode=["min", "max"])
algo = ConcurrencyLimiter(searcher, max_concurrent=4)

tuner = tune.Tuner(
    multi_objective,
    tune_config=tune.TuneConfig(
        search_alg=algo,
        num_samples=num_samples,
    ),
    param_space=search_space,
)

results = tuner.fit()

2024-12-03 18:01:23,791	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/Users/sidharrthnagappan/ray_results/multi_objective_2024-12-03_18-01-00' in 0.0104s.
2024-12-03 18:01:23,798	INFO tune.py:1041 -- Total run time: 23.50 seconds (23.47 seconds for the tuning loop).


In [25]:
print("Best hyperparameters for loss found were: ", results.get_best_result("loss", "min").config)
print("Best hyperparameters for gain found were: ", results.get_best_result("gain", "max").config)

Best hyperparameters for loss found were:  {'steps': 100, 'width': 19.14733956576996, 'height': -73.12127441630184, 'activation': 'tanh'}
Best hyperparameters for gain found were:  {'steps': 100, 'width': 2.575578383941741, 'height': 61.66395148812127, 'activation': 'relu'}
